# Satellite Link Analysis tool

In [3]:
import math
import numpy as np

## Let's define a few constants

In [4]:
EARTH_RADIUS = 6378*1000 # m
GEO_RADIUS = 42174*1000 # m, distance between geostationary satellite and center of the EARTH
GEO_ALTITUDE = GEO_RADIUS - EARTH_RADIUS

## Let's define utility funtions

In [5]:
'''Let's define utility functions'''

def to_deg(rad):
    return rad*180/math.pi

def to_rad(deg):
    return deg*math.pi/180

def to_dB(decimal):
    return 10*math.log10(decimal)

def to_decimal(dB):
    return 10**(dB/10)

def cos_deg(deg):
    return math.cos(to_rad(deg))

def sin_deg(deg):
    return math.sin(to_rad(deg))

## Now let's define getter functions for single quantities

In [6]:
def get_alpha_beta(elevation_angle):
    """
    Compute the angle between the (satellite-earth center) line and the (satellite-earth surface) line.
    The latter has an elevation_angle given with respect to the earth tangent.
    All angles abre computed in degrees
    """
    
    alpha = to_deg(math.asin(EARTH_RADIUS * sin_deg(90. + elevation_angle) / GEO_RADIUS))
    beta = 180. - alpha - 90. - elevation_angle
    return alpha, beta
    

In [7]:
def get_range(elevation_angle, sat_altitude=GEO_ALTITUDE):
    '''
    Computes the range (km) of a satellite with a given elevation angle.
    It is possible to define a sat_altitude in case the satellite is not geostationary'''
    
    alpha, beta = get_alpha_beta(elevation_angle)
    return math.sqrt( (sat_altitude+EARTH_RADIUS)**2 + EARTH_RADIUS**2 - 2*EARTH_RADIUS*(EARTH_RADIUS+sat_altitude) * cos_deg(180-(90+elevation_angle)-alpha)) 

In [8]:
def get_covered_area(elevation_angle, sat_altitude=GEO_ALTITUDE):
    
    '''
    Computes the approximation of the covered area (km^2)of a satellite with a given elevation angle
    '''
    
    alpha, beta = get_alpha_beta(elevation_angle)
    
    if (alpha < 10.):
        the_range = get_range(elevation_angle, sat_altitude)
        r = sin_deg(alpha) * the_range
        return math.pi*r*r
    
    raise ValueError("The angle Theta/Alpha is over 10° so the approximation made by this function will be incorrect.")

In [9]:
def get_orbit_period(orbit_radius):
    '''
    Combutes the orbital period of a satellite in hours'''
    result = 24.*math.pow(orbit_radius/GEO_RADIUS, 1.5)
    pretty_print(result)
    return result

def pretty_print(hour):
    h = int(np.floor(hour))
    frac_h = hour-h
    
    minutes = frac_h * 60
    m = int(np.floor(minutes))
    frac_m = minutes - m
    
    seconds = frac_m * 60
    s = int(np.floor(seconds))
    
    h_str = "hours," if h > 1 else "hour,"
    m_str = "minutes," if m > 1 else "minute,"
    s_str = "seconds." if s > 1 else "second."
    
    print("Duration :", h, h_str, m, m_str, s, s_str)

In [10]:
def get_path_loss(wave_len, the_range, dB=True):
    '''By default the result is in dB'''
    
    result = (wave_len/(4.*math.pi*the_range))**2
    
    if not dB:
        return result
    
    return to_dB(result)
    

In [11]:
def get_anten_area(dish_diameter):
    """Computes the area of a dish antenna given a specified diameter.
    Parameter and output are in meters"""
    return math.pi*dish_diameter*dish_diameter/4.

In [12]:
def get_lambda(transmit_freq, c=3.0*pow(10,8)):
    """Computes the wavelength of a wave with given frequency"""
    return c/transmit_freq

In [13]:
def get_gain_directive(anten_area, wave_len, dB=True):
    """Computes the gain of a directive antenna with given area and knowing the wave length
    of the incoming signal.
    By default the output is in dBi"""
    
    result = 4*math.pi*anten_area/(wave_len*wave_len)
    
    if not dB:
        return result
    
    return to_dB(result)

In [14]:
def get_gain_parabolic(diameter, wavelen, epsilon=0.55, dB= True):
    """Computes the gain of a parabolic dish antenna with given diameter
    and knowing the wave length of the incoming signal.
    By default the output is in dBi"""
    
    result = epsilon * (math.pi*diameter/wavelen)**2
    
    if not dB:
        return result
    
    return to_dB(result)
    

In [15]:
def get_beamwidth_parabolic(diameter, wavelen, epsilon=0.55, deg=True):
    """Computes the beamwidth of a parabolic dish antenna with a given diameter
    and given the signal wavelength.
    By default the output is in degrees"""
    result = wavelen/(diameter*math.sqrt(epsilon)) 
    
    if not deg:
        return result
    
    return to_deg(result)

In [16]:
def get_EIRP(PT,GT, in_dB=True):
    """Computes the effective isotropic radiated power of a transmitter,
    given its power PT and gain GT. 
    If in_dB== True then PT and GT must be both in dB system, and the output is in dBW.
    Else, PT and GT must be in decimal and the output is in W"""
    
    if in_dB:
        return PT + GT
    
    return PT*GT

In [17]:
def get_Aeff(wavelen, isoG=1.):
    """Computes the effective area with the relative gain to an isotropic radiator"""
    return wavelen*wavelen/(4*math.pi)

In [18]:
def get_flux_density(PR, wavelen, isoG=1., in_dB=True):
    """Computes the flux density with the relative gain to an isotropic radiator.
    If in_dB == True then PR must be in dBW and the output is in dBW/m^2.
    Else, PR must be in W and the output is in W/m^2 """
    
    Aeff = get_Aeff(wavelen, isoG=isoG)
    
    if not in_dB:
        return PR/Aeff
    
    return PR - to_dB(Aeff)

In [19]:
def get_received_power(PT, GT, GR, wavelen, the_range, in_dB=True):
    """Computes the received power for an antenna.
    If in_dB==True then PT, GT, GR must be in db and the output is in dBW
    Else, PT, GT, GR MUST NOT be in dB and the output is in W
    """
    
    const = (wavelen/(4*math.pi*the_range))**2
    
    if not in_dB:
        return PT*GT*GR*const # W
    
    return PT + GT + GR + to_dB(const) # dBW

def get_received_power_EIRP(EIRP, GR, wavelen, the_range, in_dB=True):
    """Computes the received power for an antenna.
    If in_dB==True then EIRP, GR must be in db and the output is in dBW
    Else, PT, GT, GR MUST NOT be in dB and the output is in W
    """
    
    const = (wavelen/(4*math.pi*the_range))**2
    
    if not in_dB:
        return EIRP*GR*const # W
    
    return EIRP + GR + to_dB(const) # dBW
    

In [20]:
def get_figure_of_merit(GR, T, in_dB=True):
    """Computes the ratio GR / T for a receiver.
    If in_dB==True then GR and T must be in db and the output is in dB/K
    Else, GR and T MUST NOT be in dB and the output is in K^-1
    """
    
    if not in_dB:
        return GR/T
    
    return GR - T

In [21]:
def get_OFD(EIRP, Lp, uplink_losses, wavelen,  in_dB=True):
    """Computes the Operating Flux Density for a receiver.
    This corresponds to the flux density once the IBO has been applied.
    If IBO was 0 dB then we are operating at maximum power
    If in_dB==True then EIRP, Lp and lossens must be in db and the output is in dBW/m^2
    Else, EIRP, Lp and losses MUST NOT be in dB and the output is in W/m^2
    
    Lp must be a negative dB value
    Losses must be positive dB values
    """
    
    gain = 1./get_Aeff(wavelen)
    
    if in_dB:
        return EIRP + Lp - uplink_losses + to_dB(gain)
    
    return EIRP*Lp*gain/uplink_losses
    

In [22]:
def get_noise_density(nois_temp, in_dB=True):
    """
    Computes the noise density N0 knowing the noise temperatur
    If in_dB is true then nois_temp must be in dbK and the output is in dBW/Hz
    Else, nois_temp must be in K and the output is in W/Hz
    """
    
    k = 1.38*pow(10,-23)
    
    if in_dB:
        return nois_temp + to_dB(k)
    
    return nois_temp*nois_temp

In [23]:
def get_C_over_N0(PR, N0, in_dB=True):
    """
    Computes the ration C/N0 which corresponds to C/N multiplied by the bandwidth
    If in_dB is true then PR must be in dBW and N0 in dBW/Hz, and the output is in dBHz
    Else, PR must be in W and N0 in W/Hz and the output is in Hz"""
    
    if in_dB:
        return PR-N0
    
    return PR/N0

In [24]:
def get_C_over_T(C_over_N0, in_dB=True):
    """Computes the ratio C/T knowing C/N0
    If in_dB is True then C_over_N0 must be in dBHz and the output is in dBW/K
    Else, C_over_N0 must be in Hz and the output is in W/K"""
    
    k = 1.38*pow(10,-23)
    
    if in_dB:
        return C_over_N0 + to_dB(k)
    
    return C_over_N0 * k

In [25]:
def get_margin_left(data_R, C_over_N0, req_Eb_over_N0, losses, op_margin, in_dB=True):
    
    """Computes the margin left with specified values
    If in_dB is True then all parameters must be in dB system and so will be the output
    Else, all parameters MUST NOT be in dB system."""
    
    if in_dB:
        return C_over_N0 - (req_Eb_over_N0+losses+op_margin) - data_R
    
    return C_over_N0 / (req_Eb_over_N0*losses*op_margin*data_R)

In [26]:
def get_C_over_N_alldB(PT, GT, GR,  B, Lp, T= to_dB(500), LA=0., LR=0., LM=0.):
    """Computes the ratio C/N.
    Every parameter must be in dB system.
    
    NOTE : LA, LR and LM must be positive dB values
    NOTE : Lp must be a negative dB value"""
    
    if LA<0 or LR<0 or LM<0:
        raise ValueError("LA, LR and LM must be positive dB values.")
        
    if Lp > 0:
        raise ValueError("The path loss Lp must be a negative dB value")
    
    # Boltzmann constants in dB
    k = to_dB(1.38*math.pow(10,-23))
    
    return PT+GT+GR+Lp - (LA+LR+LM+k+T+B)

## Let's define more complete functions to analyze the link

In [27]:
def oracle(trans_diam=None,
           rcv_diam = None,
           freq=None,
           wavelen=None,
           elevation_angle = None,
           the_range = None,
           PT=None,
           Lp = None,
           GT=None,
           GR=None,
           G1m2 = None,
           link_losses =0.,
           OFD = None,
           SFD=None,
           IBO=0.,
           OBO=0.,
           flux=None,
           G_over_T=None,
           nois_temp=None,
           N0=None,
           PR=None,
           band=None,
           C_over_N=None,
           C_over_N0=None,
           C_over_T=None,
           data_R=None,
           op_margin=None,
           req_Eb_over_N0=None,
           EIRP=None):
    
    """Global oracle, outputs every thing we can know about the system"""

    # The data rate, the transmit power, the bandwidth and the noise temperature MUST NOT be in dB system
    data_R_dB = to_dB(data_R)
    PT_dB = to_dB(PT)
    band_dB = to_dB(band)
    nois_temp_dB = to_dB(nois_temp)
    
    
    if the_range is None and elevation_angle is not None:
        the_range = get_range(elevation_angle)
        print("Range =", the_range, "m")
    
    if freq is not None and wavelen is None:
        wavelen = get_lambda(freq)
        print("Wavelength =", wavelen, "m")
        
    if Lp is None and wavelen is not None and the_range is not None:
        Lp = get_path_loss(wavelen, the_range)
        print("Path loss Lp =", Lp, "dB")
        
    print()
        
    if G1m2 is None and wavelen is not None:
        G1m2 = to_dB(1./get_Aeff(wavelen))
        print("Gain of 1 m^2 =", G1m2, "dBW/m^2")
    
    if GT is None and trans_diam is not None and wavelen is not None:
        GT = get_gain_parabolic(trans_diam, wavelen)
        print("GT =", GT, "dBi")
    
    if GR is None and rcv_diam is not None and wavelen is not None:
        GR = get_gain_parabolic(rcv_diam, wavelen)
        print("GR =", GR, "dBi")
        
    if G_over_T is None and GR is not None and nois_temp_dB is not None:
        G_over_T = get_figure_of_merit(GR, nois_temp_dB)
        print("G/T =", G_over_T, "dB/K")
        
    if PR is None and PT_dB is not None and GT is not None and GR is not None and wavelen is not None and the_range is not None:
        PR = get_received_power(PT_dB, GT, GR, wavelen, the_range) -OBO
        print("PR =", PR, "dBW")
        
    if PR is None and EIRP is not None and GR is not None and wavelen is not None and the_range is not None:
        PR = get_received_power_EIRP(EIRP, GR, wavelen, the_range) - OBO
        print("PR =", PR, "dBW")
    
    print()
        
    if N0 is None and nois_temp_dB is not None:
        N0 = get_noise_density(nois_temp_dB)
        print("N0 =", N0, "dBW/Hz")
        
    if C_over_N0 is None and PR is not None and N0 is not None:
        C_over_N0 = get_C_over_N0(PR,N0)
        print("C/N0 =", C_over_N0, "dBHz")
        
    if C_over_T is None and C_over_N0 is not None:
        C_over_T = get_C_over_T(C_over_N0)
        print("C/T =", C_over_T, "dBW/K")
        
    if C_over_N is None and PT_dB is not None and GT is not None and GR is not None and band_dB is not None and Lp is not None and nois_temp_dB is not None:
        C_over_N = get_C_over_N_alldB(PT_dB, GT, GR, band_dB, Lp, LA=link_losses, T=nois_temp_dB)
        print("C/N =", C_over_N, "dB")
        
    if C_over_N0 is not None and req_Eb_over_N0 is not None and link_losses is not None and op_margin is not None and data_R_dB is not None:
        print("Margin left =", get_margin_left(data_R_dB, C_over_N0, req_Eb_over_N0, link_losses, op_margin))
        
    print()
        
    if flux is None and PR is not None and wavelen is not None:
        flux = get_flux_density(PR, wavelen)
        print("Flux density =", flux, "dBW/m^2")
        
        
    if EIRP is None and GT is not None and PT_dB is not None:
        EIRP = get_EIRP(PT_dB, GT)
        print("EIRP =", EIRP, "dBW")
        
    if OFD is None and EIRP is not None and Lp is not None and wavelen is not None :
        OFD = get_OFD(EIRP, Lp, link_losses, wavelen)
        print("OFD =", OFD, "dBW/m^2")
        
        if SFD is None and IBO is not None:
            print("SFD =", OFD+IBO, "dBW/m^2")
        
        if SFD is not None:
            print("IBO =", SFD-OFD, "dB")
        
    
        
    

In [31]:
oracle(freq=1800*pow(10,6),
       elevation_angle=30.,
       PT=50,
       trans_diam = 4.,
       rcv_diam=2.,
       link_losses=0.,
       
       SFD=-110, #dBW/m^2
       IBO = None,
       EIRP=None, # The EIRP concept includes the concept of transmitting antenna and transmitting power
       nois_temp=300,
       op_margin = 0.,
       req_Eb_over_N0 = None,
       data_R=100*pow(10,3),
       band=100*pow(10,3))

Range = 38621729.63965111 m
Wavelength = 0.16666666666666666 m
Path loss Lp = -189.28385667549892 dB

Gain of 1 m^2 = 26.555123647893836 dBW/m^2
GT = 34.95084918305724 dBi
GR = 28.93024926977761 dBi
G/T = 4.159036722580986 dB/K
PR = -108.41305817930389 dBW

N0 = -203.829996588791 dBW/Hz
C/N0 = 95.41693840948712 dBHz
C/T = -133.18427072650053 dBW/K
C/N = 45.416938409487116 dB

Flux density = -81.85793453141005 dBW/m^2
EIRP = 51.940549226417424 dBW
OFD = -110.78818380118766 dBW/m^2
IBO = 0.7881838011876567 dB


In [35]:
oracle(freq=2000*pow(10,6),
       elevation_angle=30.,
       PT=100,
       trans_diam = 2.,
       rcv_diam=0.6,
       link_losses=6.,
       G_over_T = None,
       SFD=-110, #dBW/m^2
       IBO = None,
       EIRP=None, # The EIRP concept includes the concept of transmitting antenna and transmitting power
       nois_temp=300,
       op_margin = 6.,
       req_Eb_over_N0 = 10,
       data_R=100*pow(10,3),
       OBO = 0,
       band=100*pow(10,3))

Range = 38621729.63965111 m
Wavelength = 0.15 m
Path loss Lp = -190.19900648671242 dB

Gain of 1 m^2 = 27.47027345910734 dBW/m^2
GT = 29.84539908099112 dBi
GR = 19.387824175384363 dBi
G/T = -5.383388371812263 dB/K
PR = -120.96578323033694 dBW

N0 = -203.829996588791 dBW/Hz
C/N0 = 82.86421335845407 dBHz
C/T = -145.73699577753356 dBW/K
C/N = 26.864213358454066 dB
Margin left = 10.864213358454066

Flux density = -93.49550977122959 dBW/m^2
EIRP = 49.845399080991115 dBW
OFD = -118.88333394661396 dBW/m^2
IBO = 8.88333394661396 dB
